In [1]:
#----------------------------------------------------------------------------#
# Import Packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [2]:
#----------------------------------------------------------------------------#
# Funktionen zur Überprüfung von Relationen (>=,=,<=) in Arrays
def CheckForLessEq(vec, val): 
    # Ausgabe: 1 = "Alle Werte von list <= 0", 0 = "Ein Wert von list > 0"
    return(all(x <= val for x in vec)) 

def CheckForEq(vec, val): 
    # Ausgabe: 1 = "Alle Werte von list = 0", 0 = "Ein Wert von list != 0"
    return(all(x == val for x in vec))

def CheckForGreaterEq(vec, val): 
    # Ausgabe: 1 = "Alle Werte von list >= 0", 0 = "Ein Wert von list < 0"
    return(all(x >= val for x in vec)) 
#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#
# Funktion zum Überprüfen, ob ein Tripel (x,lam) ein KKT-Tripel von (QP)
def checkKKT(x,lam,Q,r,A,alpha):
    # Ausgabe: 0 = (x,lam) kein KKT-Tripel, 1 = (x,lam) KKT-Tripel
    
    AT =  np.transpose(A); 
    check = 0;
    if ((CheckForEq(Q.dot(x) + r + AT.dot(lam),0))
        &(CheckForGreaterEq(lam, 0))
        &(CheckForLessEq(g(x,A,alpha), 0))
        &(np.dot(lam,g(x,A,alpha))== 0)):
        
        check = 1;
        
    return check; 
#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#
# Funktion zur Überprüfung, welche Indizes aktiv sind
def indices_active(x,A,alpha):
    At = []; 
    m = len(A); 
    #
    for i in range(0,m): 
        if (np.dot(A[i,:],x) - alpha[i])==0: 
            At.append(i+1);
        
    At = set(At); 
    return At; 
#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#
# Definieren: Quadratische Zielfunktion f(x) = 1/2*x^{T}*Q*x + r^{T}x + c, 
#             Gradient grad_f(x) = Q*x + r
#             affin-linearen Nebenbedingungen g(x) = (g_1,...,g_m)(x) = A*x - alpha
def f(x,Q,r,c):
    Qx = Q.dot(x);  
    y= (1/2)*np.dot(x,Qx) + np.dot(r,x) + c; 
    return y; 

def grad_f(x,Q,r):
    grad = Q.dot(x) + r
    return grad; 

def g(x,A,alpha):
    y = A.dot(x) - alpha
    return y; 
#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#
# Funktion der Stragie aktiver Indizes
def active_indices_qp(Q,r,c,A,alpha,x0,lam0):
    

    # Dimensionen der Matrizen
    m = len(A);                     # A ist mxn Matrix 
    n = len(Q);                     # Q ist nxn matrix
     
    # Setzen der Werte für erste Iteration
    counter = 0; 
    x = x0;             # x = x_k
    lam = lam0;         # lam = lambda_k
    Atilde0 = indices_active(x0,A,alpha);
    Atilde = Atilde0;   # Atilde = Atilde(x_k)
    indikatorKKT = checkKKT(x,lam,Q,r,A,alpha); 
    
    # Speicherung der Iterierten
    X = np.array([]);       # Speicher der Iterierten x_k+1
    S = np.array([]);       # Speicher der Suchrichtungen s_k
    L = np.array([]);       # Speicher der Lagrange-Multiplikatoren lam_k+1
    AA = [];                # Speicher der aktiven Indexmengen
    
    X = np.append(X,x); 
    L = np.append(L,lam); 
    AA.append(list(Atilde0));
    
    # Beginn: Schleife
    while indikatorKKT == 0: 
        
        l = len(Atilde);
    
        A_k = np.zeros([l,n]);      # Definieren der Matrix A_k lxn Matrix
        Atilde_cpy = Atilde.copy(); 
    
        # Konstruktion von A_k    
        for i in range(0,l): 
            j = min(Atilde_cpy); 
            A_k[i,:] = A[j-1,:]; 
            Atilde_cpy.remove(j);     
        A_kT = np.transpose(A_k); 
    
        # Konstruktion von Sattelpunktmatrix S
        Sattelmatrix = np.zeros([n+l,n+l]);
        Sattelmatrix[0:n,0:n] = Q; 
        Sattelmatrix[0:n,n:(n+l)] = A_kT;
        Sattelmatrix[n:(n+l),0:n] = A_k; 
    
        v = np.reshape(np.zeros([1,n+l]),-1);
        v[0:n]= - grad_f(x,Q,r); 
    
        # Lösen des LGS
        z = np.linalg.solve(Sattelmatrix,v); 
    
        # Lösung des LGS z = (s_{k+1},lamtilde_{k+1})
        s = z[0:n];
        lamtilde = z[n:n+l]; 
    
        # Kontruktion von lam (lambda_k+1)
        lam = np.reshape(np.zeros([1,m]),-1); 
        Atilde_cpy =  Atilde.copy(); 
        for i in range(0,l):
            j = min(Atilde_cpy); 
            lam[j-1] = lamtilde[i]; 
            Atilde_cpy.remove(j); 
        
        # Fallunterscheidungen   
        # Fall 1    
        if CheckForEq(s,0):     
        
            # Fall 1a
            if CheckForGreaterEq(lam, 0):
                y = x; 
                Atilde = Atilde;
                
            # Fall 1b
            else:
                j = np.argmin(lam)+1;
                y = x; 
                Atilde.remove(j);
                
        # Fall 2        
        else:
        
            # Fall 2a
            if CheckForLessEq(g(x+s,A,alpha), 0): 
                y = x + s;
                Atilde = Atilde;
                
            # Fall 2b
            else: 
                indices_ = set({}); 
                u = []; 
                for k in range(1,m+1):
                    if (k not in Atilde) & (np.dot(A[k-1,:],s)>0):
                        indices_.add(k);
                        u.append(   (alpha[k-1] - np.dot(A[k-1,:],x))/(np.dot(A[k-1,:],s))   ); 
                     
        
                U = np.zeros([2,len(indices_)]);
                U[1,:] = u; 
                U[0,:] = list(indices_); 
                t = np.min(U[1,:]);
            
                jtemp = np.argmin(U[1,:]);
                j = U[0,jtemp]; 
            
                y = x + t*s; 
                
                Atilde.add(int(j));
        # Update x = x_{k+1}                     
        x = y;              
        AA.append(list(Atilde));
        
        # Überprüfung ob ein KKT Tripel gefunden wurde
        indikatorKKT = checkKKT(x,lam,Q,r,A,alpha);  
        
        # Speicherung 
        X = np.append(X,x);
        L = np.append(L,lam); 
        S = np.append(S,s);
        counter = counter + 1  
    
    # Ende: Schleife
    
    # Reshaping Arrays for Output
    X  = X.reshape(counter+1,n); 
    L = L.reshape(counter+1,m);
    S = np.append(S, np.reshape(np.zeros([1,n]),-1));  #?
    S = S.reshape(counter+1,n);     
    
    # Return Output
    return AA,X,L,S,counter;    
#----------------------------------------------------------------------------#

In [3]:
# Definieren der Paramter für Funktionen f und g
Q = np.array([[2,0],
              [0,2]]); 
A = np.array([[0,-1]]); 
alpha = np.array([0]);  
r = np.array([0,4]); 
c = 0; 

# Setzen der Startwerte
x0 = np.array([5,10]);
lam0 = np.array([0]);

# Aufrufen der Funktion
AAA,XX,LL,SS,itera = active_indices_qp(Q,r,c,A,alpha,x0,lam0); 

# Ausgabe der Ergebnisse
print("      Strategie Aktiver Indizes     "); 
print("-------------------------------------")
for k in range(0,itera):
    print("Iteration: ",k);
    print("x_%i      = "%(k+1), XX[k+1,:]);
    print("lam_%i    = " %(k+1), LL[k+1,:]);
    print("s_%i      = " %k, SS[k,:]);
    print("Atilde_%i = " %(k+1), set(AAA[k]));
    print("-------------------------------------");
    
print("Iteration: ",itera);
print("x_%i      = " %(itera+1), XX[itera,:]);
print("lam_%i    = " %(itera+1), LL[itera,:]);
print("s_%i      = " %itera, SS[itera,:]); 
print("Atilde_%i = " %(itera+1), set(AAA[itera]));
print("-------------------------------------");

      Strategie Aktiver Indizes     
-------------------------------------
Iteration:  0
x_1      =  [0.83333333 0.        ]
lam_1    =  [0.]
s_0      =  [ -5. -12.]
Atilde_1 =  set()
-------------------------------------
Iteration:  1
x_2      =  [0. 0.]
lam_2    =  [4.]
s_1      =  [-0.83333333  0.        ]
Atilde_2 =  {1}
-------------------------------------
Iteration:  2
x_3      =  [0. 0.]
lam_3    =  [4.]
s_2      =  [0. 0.]
Atilde_3 =  {1}
-------------------------------------
